<a href="https://colab.research.google.com/github/Arunim10/Age-Detection-of-Indian-Actors/blob/main/age_detect_stratifiedkfold_ipynb_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split,Dataset
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
# from skmultilearn.model_selection import IterativeStratification
from sklearn.model_selection import StratifiedKFold
import joblib
import time
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from zipfile import ZipFile
# file_name = '/content/drive/MyDrive/age_detection/train_DETg9GD.zip'

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('Done')

In [ ]:
# from zipfile import ZipFile
# file_name = '/content/drive/MyDrive/age_detection/test_Bh8pGW3.zip'

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('Done')

In [ ]:
# import os
# train_images_path = "/content/Train"
# train_csv = "/content/train.csv"

In [ ]:
import os
train_images_path = "../input/agedetectstratifiedkfold/train_DETg9GD/Train"
train_csv = "../input/agedetectstratifiedkfold/train_DETg9GD/train.csv"
test_images_path = "../input/agedetectstratifiedkfold/test_Bh8pGW3/Test"
test_csv = "../input/agedetectstratifiedkfold/test_Bh8pGW3/test.csv"

In [ ]:
len(os.listdir(train_images_path))

In [ ]:
import pandas as pd
train_df = pd.read_csv(train_csv)
train_df.head()

In [ ]:
test_df = pd.read_csv(test_csv)
test_df.head()

# Using GPU

In [ ]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')

def to_device(data,device):
  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

class DeviceDataLoader():
  def __init__(self,dl,device):
    self.dl = dl
    self.device = device
  
  def __iter__(self):
    for b in self.dl:
      yield to_device(b,self.device)
  
  def __len__(self):
    return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
# train_dl = DeviceDataLoader(train_dl,device)
# val_dl = DeviceDataLoader(val_dl,device)

# Models

In [ ]:
train_df.head()

In [ ]:
train_df['Class'] = train_df['Class'].map({'MIDDLE':0,'YOUNG':2,'OLD':1})
train_df.head()

In [ ]:
X,y = train_df['ID'].values,train_df['Class']
X,y

In [ ]:
# stats = ((0.3033631  ,0.33424648, 0.43405876),(0.42803708 ,0.42595481 ,0.45326769))
stats = ((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
train_tfms = tt.Compose([tt.Resize((224,224))
#                         ,tt.RandomCrop(512,padding=4,padding_mode='reflect')
                        # ,tt.CenterCrop(128)
#                          ,tt.ColorJitter(brightness=0.35, contrast=0.15, saturation=0.28, hue=0.22)
#                         ,tt.RandomRotation(90)
#                          ,tt.RandomPerspective()
#                         ,tt.RandomHorizontalFlip()
                        ,tt.RandomVerticalFlip(),
                         tt.RandomEqualize(p=0.4),
                        tt.RandomAutocontrast(0.3),
                        tt.RandomHorizontalFlip(p=0.3),
                        tt.AutoAugment(),
                        tt.ToTensor(),
                        tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize((224,224)),
                         tt.ToTensor(),
                         tt.Normalize(*stats)])
# test_tfms = tt.Compose([tt.Resize((256,256)),
#                          tt.ToTensor(),
#                          tt.Normalize(*stats)])

In [ ]:
from torchvision.utils import make_grid
def denormalize(image,means,stds):
  means = torch.tensor(means).reshape(1,3,1,1)
  print("means :",means)
  stds = torch.tensor(stds).reshape(1,3,1,1)
  print("stds :",stds)
  print("image.shape :",image.shape)
  return image*stds+means
def show_batch(dl,subset="train"):
    if (subset=="train"):
        for images, labels in dl:
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.set_xticks([]); ax.set_yticks([])
            denorm_images = denormalize(images,*stats)
            ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
            break
    else:
        for images in dl:
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.set_xticks([]); ax.set_yticks([])
            denorm_images = denormalize(images,*stats)
            ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
            break

In [ ]:
# train_ds = AgeDetectDataset(train_df, transform=train_tfms)
# train_dl = DataLoader(train_ds, 32, shuffle=True, num_workers=2, pin_memory=True)

In [ ]:
# show_batch(train_dl)

In [ ]:
from pathlib import Path

In [ ]:
train_images = {int(x.stem): x for x in Path(train_images_path).iterdir() if x.suffix == '.jpg'}

In [ ]:
test_images = {int(x.stem): x for x in Path(test_images_path).iterdir() if x.suffix == '.jpg'}

In [ ]:
test_images[25540]

In [ ]:
class AgeDetectDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df
        self.transform = transform
        self.files = test_images if is_test else train_images
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_id, img_label = row['ID'], row['Class']
        labels = torch.zeros(3)
        labels[img_label] = 1
        img = os.path.join(train_images_path,img_id)
#         img = self.files[img_id] 
#         img = Image.open(img)
        img = Image.open(img,mode='r').convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, labels

In [ ]:
torch.zeros(3)

In [ ]:
test_df.head()

In [ ]:
class AgeDetectDatasetTest(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df
        self.transform = transform
        self.files = test_images if is_test else train_images
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_id = row['ID']
        img = os.path.join(test_images_path,img_id)
#         img = self.files[img_id] 
#         img = Image.open(img)
        img = Image.open(img,mode='r').convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
batch_size = 32

In [ ]:
def get_split_dataloaders(train_df,val_df):
    
    train_ds = AgeDetectDataset(train_df, transform=train_tfms)
    val_ds = AgeDetectDataset(val_df, transform=valid_tfms)
    
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)
    
    
    train_dl = DeviceDataLoader(train_dl, device)
    val_dl = DeviceDataLoader(val_dl, device)
    
    return train_dl, val_dl

In [ ]:
def get_test_dl():
    test_ds = AgeDetectDatasetTest(test_df, transform=valid_tfms, is_test=True)
    test_dl = DataLoader(test_ds, batch_size*2, num_workers=2, pin_memory=True)
    return DeviceDataLoader(test_dl, device)

In [ ]:
a = torch.zeros((1,3))
print(a)
a[0][2] = 1
print(a)
val,out = torch.max(a,dim=1)
out

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    val,out = torch.max(output,dim=1)
#     print(val.shape)
#     print(label.shape)
    lab,label = torch.max(label,dim=1) 
    prob = val > threshold
    label = label > threshold
    # print("prob: ",prob)
    # print("label: ",label)
    TP = (prob & label).sum().float()
    TN = ((~prob) & (~label)).sum().float()
    FP = (prob & (~label)).sum().float()
    FN = ((~prob) & label).sum().float()

    precision = torch.mean(TP / (TP + FP+1e-12))
    recall = torch.mean(TP / (TP + FN+1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall+1e-12)
    return F2.mean(0)

In [ ]:
class LabelSmoothingLoss(nn.Module): 
    def __init__(self, classes, smoothing, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim)
        _,target = torch.max(target,dim=1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
#             print("1")
#             print(true_dist,true_dist.shape)
            true_dist.fill_(self.smoothing / (self.cls - 1))
#             print("2")
#             print(true_dist,true_dist.shape)
#             print(target.data.shape)
#             print(target.data.shape)
            true_dist.scatter_(1, target.data.type(torch.int64).unsqueeze(1), self.confidence) 
#             print("3")
#             print(true_dist,true_dist.shape)
#         print(torch.mean(torch.sum(-true_dist * pred, dim=self.dim)))
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
criterion = LabelSmoothingLoss(classes=3, smoothing=0.05)

In [ ]:
def accuracy(outputs,labels):
#   print(outputs)
#   print(labels)
  _,preds = torch.max(outputs,dim=1)
  _,labels = torch.max(labels,dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))

class MultilabelImageClassificationBase(nn.Module):
  def training_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = criterion(out,labels)
    return loss
  
  def validation_step(self,batch):
    images,labels = batch
    # print(images.shape)
    # print(images)
    # print(labels.shape)
    out = self(images)
    # print("OUT ")
    # # print(out)
    # print(out.shape)
    loss = criterion(out,labels)
    score = F_score(out,labels)
    acc = accuracy(out,labels)
    return {'val_loss':loss.detach(),'val_acc':acc,'val_score':score.detach()}
  
  def validation_epoch_end(self,outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()
    batch_scores = [x['val_score'] for x in outputs]
    epoch_score = torch.stack(batch_scores).mean()
    return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item(), 'val_score':epoch_score.item()}

  def epoch_end(self,epoch,result):
    print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, val_score: {:.4f}".format(
        epoch,result['lrs'][-1],result['train_loss'],result['val_loss'],result['val_acc'],result['val_score']))

# Resnet 

In [ ]:
# def conv_block(in_channels, out_channels, pool=False):
#     layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
#               nn.BatchNorm2d(out_channels), 
#               nn.ReLU(inplace=True)]
#     if pool: layers.append(nn.MaxPool2d(2))
#     return nn.Sequential(*layers)

# class ResNet9(MultilabelImageClassificationBase):
#     def __init__(self, in_channels, num_classes):
#         super().__init__()
        
#         self.conv1 = conv_block(in_channels, 64)
#         self.conv2 = conv_block(64, 128, pool=True)
#         self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
#         self.conv3 = conv_block(128, 256, pool=True)
#         self.conv4 = conv_block(256, 512, pool=True)
#         self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
#         self.classifier = nn.Sequential(nn.MaxPool2d(4), 
#                                         nn.Flatten(), 
#                                         nn.Dropout(0.2),
#                                         nn.Linear(512, num_classes))
        
#     def forward(self, xb):
#         out = self.conv1(xb)
#         # print("1")
#         # print(out.shape)
#         out = self.conv2(out)
#         # print("2")
#         # print(out.shape)
#         # print("self.res1(out) ",self.res1(out).shape)
#         out = self.res1(out) + out
#         # print("3")
#         # print(out.shape)
#         out = self.conv3(out)
#         # print("4")
#         # print(out.shape)
#         out = self.conv4(out)
#         # print("5")
#         # print(out.shape)
#         # print("self.res2(out) ",self.res2(out).shape)
#         out = self.res2(out) + out
#         # print("6")
#         # print(out.shape)
#         out = self.classifier(out)
#         # print("7")
#         # print(out.shape)
#         return out

In [ ]:
# model = models.resnext50_32x4d()
# model

In [ ]:
class Resnext50(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnext50_32x4d(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs,3)
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        
        for param in self.network.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

In [ ]:
# model = to_device(ResNet9(3, 3), device)
# model

In [ ]:
# resnet50 = models.resnet50()
# resnet50

In [ ]:
# resnet152 = models.resnet152()
# resnet152

# DENSENET121

In [ ]:
# models_densenet161 = models.densenet161()
# models_densenet161

In [ ]:
# models.densenet121()

In [ ]:
class DenseNET121(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.densenet121(pretrained=True)
        # Replace last layer
#         num_ftrs = self.network.fc.in_features
#         self.network.fc = nn.Linear(num_ftrs, 3)
        num_ftrs = self.network.classifier.in_features
        self.network.classifier = torch.nn.Linear(self.network.classifier.in_features, out_features=3)
#         self.network.classifier = nn.Sequential(
# # #                         #                 nn.AdaptiveAvgPool2d((1280,1,1)),
#                                         nn.Linear(num_ftrs, 512),    
#                                         nn.Hardswish(),
#                                         nn.BatchNorm1d(512),
#                                         nn.Dropout(0.3),
#                                         nn.Linear(512, 256),
#                                         nn.Hardswish(),
#                                         nn.BatchNorm1d(256),
#                                         nn.Dropout(0.3),
#                                         nn.Linear(256,3))
#                                         # nn.BatchNorm1d(num_features=num_ftrs),    
#                                         # nn.Linear(num_ftrs, 512),
#                                         # nn.ReLU(),
#                                         # nn.BatchNorm1d(512),
#                                         # nn.Linear(512, 128),
#                                         # nn.ReLU(),
#                                         # nn.BatchNorm1d(num_features=128),
#                                         # nn.Dropout(0.4),
#                                         # nn.Linear(128, 3),
#                                     )
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

# Efficient Net_B2

In [ ]:
# !pip install timm

In [ ]:
# import timm

In [ ]:
# models.efficientnet_b2

In [ ]:
class EfficientNet_b2(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = timm.create_model("efficientnet_b2",pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier.in_features
        self.network.classifier = nn.Linear(num_ftrs,3)
#         self.network.classifier = nn.Sequential(
# #                         #                 nn.AdaptiveAvgPool2d((1280,1,1)),
#                                         nn.Linear(num_ftrs, 512),    
#                                         nn.Hardswish(),
#                                         nn.BatchNorm1d(512),
#                                         nn.Dropout(0.3),
#                                         nn.Linear(512, 256),
#                                         nn.Hardswish(),
#                                         nn.BatchNorm1d(256),
#                                         nn.Dropout(0.3),
#                                         nn.Linear(256,3)
#                                         # nn.BatchNorm1d(num_features=num_ftrs),    
#                                         # nn.Linear(num_ftrs, 512),
#                                         # nn.ReLU(),
#                                         # nn.BatchNorm1d(512),
#                                         # nn.Linear(512, 128),
#                                         # nn.ReLU(),
#                                         # nn.BatchNorm1d(num_features=128),
#                                         # nn.Dropout(0.4),
#                                         # nn.Linear(128, 3),
#                                     )
    
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
          
        for param in self.network.classifier.parameters():
          param.require_grad = True    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

# Efficient_B3

In [ ]:
!pip install timm

In [ ]:
import timm

In [ ]:
class EfficientNet_b3(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = timm.create_model("efficientnet_b3",pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier.in_features
#         self.network.classifier = nn.Linear(num_ftrs,3)
        self.network.classifier = nn.Sequential(
#                         #                 nn.AdaptiveAvgPool2d((1280,1,1)),
                                        nn.Linear(num_ftrs, 512),    
                                        nn.Hardswish(),
                                        nn.BatchNorm1d(512),
                                        nn.Dropout(0.3),
                                        nn.Linear(512, 256),
                                        nn.Hardswish(),
                                        nn.BatchNorm1d(256),
                                        nn.Dropout(0.3),
                                        nn.Linear(256,3)
                                        # nn.BatchNorm1d(num_features=num_ftrs),    
                                        # nn.Linear(num_ftrs, 512),
                                        # nn.ReLU(),
                                        # nn.BatchNorm1d(512),
                                        # nn.Linear(512, 128),
                                        # nn.ReLU(),
                                        # nn.BatchNorm1d(num_features=128),
                                        # nn.Dropout(0.4),
                                        # nn.Linear(128, 3),
                                    )
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
          
        for param in self.network.classifier.parameters():
          param.require_grad = True    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

# Training Model

In [ ]:
@torch.no_grad()
def evaluate(model,val_loader):
  model.eval()
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']
  
def fit_one_cycle(epochs,max_lr,model,train_loader,val_loader,
                  weight_decay=0,grad_clip=None,opt_func = torch.optim.SGD):
  torch.cuda.empty_cache()
  history = []

  optimizer = opt_func(model.parameters(),max_lr,weight_decay=weight_decay,amsgrad=False)
#   sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=1, cooldown=1)
  sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=4, verbose=True, eps=1e-6)
#   sched = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr,epochs=epochs,steps_per_epoch=len(train_loader))

  for epoch in range(epochs):
    model.train()
    train_losses = []
    lrs = []
    for batch in train_loader:
      loss = model.training_step(batch)
      train_losses.append(loss)
      loss.backward()
      
      if grad_clip:
        nn.utils.clip_grad_value_(model.parameters(),grad_clip)

      optimizer.step()
      optimizer.zero_grad()

      lrs.append(get_lr(optimizer))
#       sched.step()

    result = evaluate(model,val_loader)
    sched.step(result['val_loss'])
    result['train_loss']=torch.stack(train_losses).mean().item()
    result['lrs'] = lrs
    model.epoch_end(epoch,result)
    history.append(result)
  return history

In [ ]:
# model = to_device(EfficientNet_b4(),device)

In [ ]:
def plot_scores(history):
    scores = [x['val_score'] for x in history]
    plt.plot(scores, '-x')
    plt.xlabel('epoch')
    plt.ylabel('score')
    plt.title('F1 score vs. No. of epochs');
    plt.show()

In [ ]:
def plot_accuracies(history):
  accuracies = [x['val_acc'] for x in history]
  plt.plot(accuracies,'-x')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.title('Accuracy vs No. of epochs')
  plt.show()


In [ ]:
def plot_losses(history):
  train_losses = [x.get('train_loss') for x in history]
  val_losses = [x['val_loss'] for x in history]
  plt.plot(train_losses,'-bx')
  plt.plot(val_losses,'-rx')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend(['Training','Validation'])
  plt.title('Loss vs No. of epochs')
  plt.show()


In [ ]:
def plot_lrs(history):
  lrs = np.concatenate([x.get('lrs',[]) for x in history])
#   print(lrs)
  val_losses = [x['val_loss'] for x in history]
  train_losses = [x.get('train_loss') for x in history]
  # print(len(lrs))
  # print(len(val_losses))
  # plt.plot(lrs,val_losses,marker='-rx')

  # plt.plot(lrs,train_losses,marker='-bx')
  plt.plot(lrs)
  plt.xlabel('Batch no.')
  # plt.ylabel('Losses')
  plt.ylabel('Learning Rate')
  # plt.xlabel('Learning Rate')
  plt.title('Learning Rate vs Batch no.')
  plt.show()
  # plt.title('Learning Rate vs Losses')


In [ ]:
# for train_in,val_in in skf.split(X,y):
#   train = train_df.iloc[train_in,:].reset_index()
#   val = train_df.iloc[val_in,:].reset_index()
#   print(train.shape)
#   print(val.shape)
#   random_seed = 42
#   torch.manual_seed(random_seed);
#   # print(train.shape)
#   # print(val.shape)
#   # break
#   print(train.head())
#   train_dl,val_dl = get_split_dataloaders(train,val)
#   for img,la in train_dl:
#     print(img.shape,la.shape)
#     break
#   break

In [ ]:
a = torch.from_numpy(np.array([1]))
a.shape

In [ ]:
skf = StratifiedKFold(n_splits=5)
fold = 0
model = to_device(EfficientNet_b3(),device)
for train_in,val_in in skf.split(X,y):
#     if (fold==0):
      train = train_df.iloc[train_in,:].reset_index()
      val = train_df.iloc[val_in,:].reset_index()
#       print(train.shape)
#       print(val.shape)
      random_seed = 42
      torch.manual_seed(random_seed);
      # print(train.shape)
      # print(val.shape)
      # break
      train_dl,val_dl = get_split_dataloaders(train,val)
      for m,n in train_dl:
        print(m.shape)
        break
      history = [evaluate(model,val_dl)]
      model.freeze()
      epochs = 5
      max_lr = 0.0001
      grad_clip = 1e-1
      weight_decay = 1e-5
      opt_func = torch.optim.AdamW
      # %%time
      since = time.time()
      history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                                 grad_clip=grad_clip, 
                                 weight_decay=weight_decay, 
                                 opt_func=opt_func)
      epochs = 5
      weight_decay = 1e-5
      model.unfreeze()
      # %%time
      history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                                 grad_clip=grad_clip, 
                                 weight_decay=weight_decay, 
                                 opt_func=opt_func)
      print(" Training time for fold ",fold," is ",(time.time() - since)/60," min")

      plot_scores(history)
      plt.show()
      plot_accuracies(history)
      plt.show()
      plot_losses(history)
      plt.show()
      plot_lrs(history)
      plt.show()

      densenet121_fold = {
          'model': model,
          'epochs_freeze' : 5,
          'epochs_unfreeze' : 5,
          'max_lr' :0.0001,
          'grad_clip' : 1e-4,
          'weight_decay' : 1e-5/1e-5,
          'opt_func' : torch.optim.AdamW,
          'scheduler' : "ReduceLRonPlateau",
          'time': (time.time() - since)/60
      }
      model_file_name = "Densenet121_"+str(fold)+".joblib"
      joblib.dump(densenet121_fold,model_file_name)
      fold+=1

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working")

In [ ]:
plot_scores(history)
plt.show()
plot_accuracies(history)
plt.show()
plot_losses(history)
plt.show()
plot_lrs(history)
plt.show()

In [ ]:
# import jovian
# jovian.commit(project="age-detection-latest-74f18")

# Predictions 

In [ ]:
def predict_single(image):
    xb = image.unsqueeze(0)
    xb = to_device(xb, device)
    preds = model(xb)
    prediction = preds[0]
    print("Prediction: ", prediction)
    show_sample(image, prediction)
    
@torch.no_grad()
def predict_dl(dl, model):
    torch.cuda.empty_cache()
    batch_probs = []
    for xb in tqdm(dl):
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    return batch_probs

In [ ]:
densenet121_model_0 = joblib.load("../input/densenet121-models/Densenet121_0_new.joblib")['model']
densenet121_model_1 = joblib.load("../input/densenet121-models/Densenet121_1_new.joblib")['model']
densenet121_model_2 = joblib.load("../input/densenet121-models/Densenet121_2_new.joblib")['model']
densenet121_model_3 = joblib.load("../input/densenet121-models/Densenet121_3_new.joblib")['model']
densenet121_model_4 = joblib.load("../input/densenet121-models/Densenet121_4_new.joblib")['model']

In [ ]:
effnetb2_model_0 = joblib.load("../input/effnetb2-label-smooth/Effnetb2_label_smooth_0.joblib")['model']
effnetb2_model_1 = joblib.load("../input/effnetb2-label-smooth/Effnetb2_label_smooth_1.joblib")['model']
effnetb2_model_2 = joblib.load("../input/effnetb2-label-smooth/Effnetb2_label_smooth_2.joblib")['model']
effnetb2_model_3 = joblib.load("../input/effnetb2-label-smooth/Effnetb2_label_smooth_3.joblib")['model']
effnetb2_model_4 = joblib.load("../input/effnetb2-label-smooth/Effnetb2_label_smooth_4.joblib")['model']

In [ ]:
effnetb3_model_0 = joblib.load("../input/effnetb3-label-smooth/EfficientNetB3_label_smooth_0.joblib")['model']
effnetb3_model_1 = joblib.load("../input/effnetb3-label-smooth/EfficientNetB3_label_smooth_1.joblib")['model']
effnetb3_model_2 = joblib.load("../input/effnetb3-label-smooth/EfficientNetB3_label_smooth_2.joblib")['model']
effnetb3_model_3 = joblib.load("../input/effnetb3-label-smooth/EfficientNetB3_label_smooth_3.joblib")['model']
effnetb3_model_4 = joblib.load("../input/effnetb3-label-smooth/EfficientNetB3_label_smooth_4.joblib")['model']

In [ ]:
resnext50_model_0 = joblib.load("../input/resnext50-32x4d-models/Resnext50_32x4d_0.joblib")['model']
resnext50_model_1 = joblib.load("../input/resnext50-32x4d-models/Resnext50_32x4d_1.joblib")['model']
resnext50_model_2 = joblib.load("../input/resnext50-32x4d-models/Resnext50_32x4d_2.joblib")['model']
resnext50_model_3 = joblib.load("../input/resnext50-32x4d-models/Resnext50_32x4d_3.joblib")['model']
resnext50_model_4 = joblib.load("../input/resnext50-32x4d-models/Resnext50_32x4d_4.joblib")['model']

In [ ]:
from tqdm import tqdm

In [ ]:
# test_ds = AgeDetectDatasetTest(test_df, transform=valid_tfms, is_test=True)
# test_dl = DataLoader(test_ds, batch_size*2, num_workers=2, pin_memory=True)
# show_batch(test_dl,"test")

In [ ]:
test_dl = get_test_dl()
densenet121_preds_0 = predict_dl(test_dl,densenet121_model_0)
densenet121_preds_1 = predict_dl(test_dl,densenet121_model_1)
densenet121_preds_2 = predict_dl(test_dl,densenet121_model_2)
densenet121_preds_3 = predict_dl(test_dl,densenet121_model_3)
densenet121_preds_4 = predict_dl(test_dl,densenet121_model_4)

In [ ]:
effnetb2_preds_0 = predict_dl(test_dl,effnetb2_model_0)
effnetb2_preds_1 = predict_dl(test_dl,effnetb2_model_1)
effnetb2_preds_2 = predict_dl(test_dl,effnetb2_model_2)
effnetb2_preds_3 = predict_dl(test_dl,effnetb2_model_3)
effnetb2_preds_4 = predict_dl(test_dl,effnetb2_model_4)

In [ ]:
effnetb3_preds_0 = predict_dl(test_dl,effnetb3_model_0)
effnetb3_preds_1 = predict_dl(test_dl,effnetb3_model_1)
effnetb3_preds_2 = predict_dl(test_dl,effnetb3_model_2)
effnetb3_preds_3 = predict_dl(test_dl,effnetb3_model_3)
effnetb3_preds_4 = predict_dl(test_dl,effnetb3_model_4)

In [ ]:
resnext50_preds_0 = predict_dl(test_dl,resnext50_model_0)
resnext50_preds_1 = predict_dl(test_dl,resnext50_model_1)
resnext50_preds_2 = predict_dl(test_dl,resnext50_model_2)
resnext50_preds_3 = predict_dl(test_dl,resnext50_model_3)
resnext50_preds_4 = predict_dl(test_dl,resnext50_model_4)

In [ ]:
densenet121_preds = []
effnetb2_preds = []
effnetb3_preds = []
resnext50_preds = []
densenet121_preds.append(densenet121_preds_0)
densenet121_preds.append(densenet121_preds_1)
densenet121_preds.append(densenet121_preds_2)
densenet121_preds.append(densenet121_preds_3)
densenet121_preds.append(densenet121_preds_4)

effnetb2_preds.append(effnetb2_preds_0)
effnetb2_preds.append(effnetb2_preds_1)
effnetb2_preds.append(effnetb2_preds_2)
effnetb2_preds.append(effnetb2_preds_3)
effnetb2_preds.append(effnetb2_preds_4)

effnetb3_preds.append(effnetb3_preds_0)
effnetb3_preds.append(effnetb3_preds_1)
effnetb3_preds.append(effnetb3_preds_2)
effnetb3_preds.append(effnetb3_preds_3)
effnetb3_preds.append(effnetb3_preds_4)

resnext50_preds.append(resnext50_preds_0)
resnext50_preds.append(resnext50_preds_1)
resnext50_preds.append(resnext50_preds_2)
resnext50_preds.append(resnext50_preds_3)
resnext50_preds.append(resnext50_preds_4)

In [ ]:
densenet121_prediction_cv = torch.stack(densenet121_preds).mean(axis=0)
effnetb2_prediction_cv = torch.stack(effnetb2_preds).mean(axis=0)
effnetb3_prediction_cv = torch.stack(effnetb3_preds).mean(axis=0)
resnext50_prediction_cv = torch.stack(resnext50_preds).mean(axis=0)

In [ ]:
_,densenet121_predictions = torch.max(densenet121_prediction_cv,dim=1)
_,effnetb2_predictions = torch.max(effnetb2_prediction_cv,dim=1)
_,effnetb3_predictions = torch.max(effnetb3_prediction_cv,dim=1)
_,resnext50_predictions = torch.max(resnext50_prediction_cv,dim=1)

In [ ]:
# predic = []
# predic.append(densenet121_predictions)
# predic.append(effnetb2_predictions)
# predic.append(effnetb3_predictions)
# predic.append(resnext50_predictions)
# predic

In [ ]:
# pred_hardvoting = []
# for i in range(len(effnetb2_predictions)):
#     cnt0 = 0
#     cnt1 = 0
#     cnt2 = 0
#     for j in predic:
#         if (j[i]==0):
#             cnt0+=1
#         elif (j[i]==1):
#             cnt1+=1
#         else:
#             cnt2+=1
#     maxi = max(cnt0,max(cnt1,cnt2))
#     if (cnt0==maxi):
#         pred_hardvoting.append(0)
#     elif (cnt1==maxi):
#         pred_hardvoting.append(1)
#     else:
#         pred_hardvoting.append(2)
# len(pred_hardvoting)

In [ ]:
# pred_hardvoting[:5]

In [ ]:
# pred_df = pd.DataFrame(pred_hardvoting,columns=["Class"])
# pred_df.head()

In [ ]:
# submission_df = pd.concat([pred_df,test_df],axis=1)
# submission_df.head()

In [ ]:
# submission_df['Class'] = submission_df['Class'].map({0:'MIDDLE',1:'OLD',2:'YOUNG'})
# submission_df.head()

In [ ]:
# submission_df.to_csv('all_submission-hard_voting_age-detection.csv',index=False)

In [ ]:
all_preds = []
all_preds.append(densenet121_prediction_cv)
all_preds.append(effnetb2_prediction_cv)
all_preds.append(effnetb3_prediction_cv)
all_preds.append(resnext50_prediction_cv)

In [ ]:
torch.stack(all_preds).shape

In [ ]:
allpred_cv = torch.stack(all_preds).mean(axis=0)
allpred_cv.shape

In [ ]:
_,pred = torch.max(allpred_cv,dim=1)
pred.shape

In [ ]:
pred_df = pd.DataFrame(pred,columns=["Class"])
pred_df.head()

In [ ]:
submission_df = pd.concat([pred_df,test_df],axis=1)
submission_df.head()

In [ ]:
submission_df['Class'] = submission_df['Class'].map({0:'MIDDLE',1:'OLD',2:'YOUNG'})
submission_df.head()

In [ ]:
submission_df.to_csv('all_submission-age-detection.csv',index=False)